## Importing Modules

In [11]:
import os
import pathlib
import numpy as np
import pandas as pd
import nltk

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config, GPT2Tokenizer
from transformers import get_linear_schedule_with_warmup

from tqdm.auto import tqdm
import random
import datetime
import time
import statistics
from nltk.translate.bleu_score import sentence_bleu
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available.")

GPU is available!


In [3]:
MAIN_PATH = str(pathlib.Path().resolve())
DATASET_PATH = MAIN_PATH + '\\datasets'
MODEL_PATH = MAIN_PATH + '\\models'

In [4]:
models = os.listdir(MODEL_PATH)
models

['bert-base-cased',
 'bert-base-multilingual-cased',
 'bert-base-uncased',
 'bert-large-cased',
 'bert-large-uncased',
 'flan-t5-base',
 'flan-t5-large',
 'flan-t5-small',
 'gpt2',
 'gpt2-large',
 'gpt2-medium']

In [5]:
model_path = MODEL_PATH + '\\' + models[8]
model_path

'D:\\Python\\LLM_Environment\\models\\gpt2'

In [6]:
configuration = GPT2Config.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path, config=configuration)

## Import Dataset

In [7]:
filenames = os.listdir(DATASET_PATH)
filenames

['Html.csv', 'Recipes.csv', 'Recipes_1000.csv']

In [8]:
file_path = DATASET_PATH + '\\' + filenames[0]
file_path

'D:\\Python\\LLM_Environment\\datasets\\Html.csv'

In [9]:
df = pd.read_csv(file_path)
df.shape

(6712, 2)

In [10]:
df.head()

,Bad_Practices,Good_Practices
0,<table alt=header>Title</table>,<table alt='header'>Title</table>
1,<tr>Content,<tr>Content</tr>
2,<h2 src='description'>Content,<h2 src='description'>Content</h2>
3,<table>Link,<table>Link</table>
4,<img src='description'>,<img src='description' alt=''>


In [12]:
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# Set the pad_token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Load your dataset in a CSV file
dataset = load_dataset("csv", data_files=file_path)

# Split the dataset into training and validation sets
train_data = dataset["train"].select([i for i in range(len(dataset["train"])) if i % 10 != 0])  # Use 90% of the data for training
val_data = dataset["train"].select([i for i in range(len(dataset["train"])) if i % 10 == 0])  # Use 10% of the data for validation

# Tokenize the input and target sequences
def tokenize_function(examples):
    inputs = tokenizer(examples['Bad_Practices'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    labels = tokenizer(examples['Good_Practices'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    return {'input_ids': inputs['input_ids'], 'labels': labels['input_ids']}

# Apply tokenization to the datasets
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

Generating train split: 6712 examples [00:00, 117370.64 examples/s]
Map: 100%|██████████| 672/672 [00:00<00:00, 2401.37 examples/s]


In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./model',
    overwrite_output_dir=True,
    num_train_epochs=0.5,
    per_device_train_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    logging_steps=100,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Fine-tune the model
trainer.train()

  0%|          | 0/1510 [00:00<?, ?it/s]d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  7%|▋         | 100/1510 [00:16<03:55,  5.98it/s]

{'loss': 0.1854, 'grad_norm': 0.503948986530304, 'learning_rate': 4.668874172185431e-05, 'epoch': 0.03}



  7%|▋         | 101/1510 [00:32<1:49:45,  4.67s/it]

{'eval_loss': 0.023037223145365715, 'eval_runtime': 14.9854, 'eval_samples_per_second': 44.844, 'eval_steps_per_second': 5.605, 'epoch': 0.03}


 13%|█▎        | 200/1510 [00:49<03:41,  5.92it/s]  

...

                                                   
 99%|█████████▉| 1500/1510 [08:32<00:01,  5.36it/s]

{'eval_loss': 0.0163738876581192, 'eval_runtime': 16.3005, 'eval_samples_per_second': 41.226, 'eval_steps_per_second': 5.153, 'epoch': 0.5}


100%|██████████| 1510/1510 [08:39<00:00,  2.91it/s]

{'train_runtime': 519.3529, 'train_samples_per_second': 5.815, 'train_steps_per_second': 2.907, 'train_loss': 0.030614700903560943, 'epoch': 0.5}


TrainOutput(global_step=1510, training_loss=0.030614700903560943, metrics={'train_runtime': 519.3529, 'train_samples_per_second': 5.815, 'train_steps_per_second': 2.907, 'total_flos': 789101936640000.0, 'train_loss': 0.030614700903560943, 'epoch': 0.5})

In [18]:
#trainer.save_model()

In [19]:
# Inference Example
example_input = "Heading"
input_ids = tokenizer(example_input, return_tensors="pt")["input_ids"].to(device)
attention_mask = torch.ones(input_ids.shape, device=device)
output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, top_k=50, top_p=0.95, attention_mask=attention_mask)

# Decode and print the corrected HTML code
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Original HTML code:", example_input)
print("Corrected HTML code:", decoded_output)

d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Original HTML code: Heading
Corrected HTML code: Heading>List Item</htable>
